In [2]:
pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 2.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import xgboost as xgb
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.svm import SVR
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Set random seed for reproducibility
np.random.seed(42)

def validate_smiles(smiles):
    """Validate SMILES string and return RDKit molecule object"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return mol
    except:
        return None

In [6]:
def get_morgan_fingerprint(mol, radius=2, nBits=1024):
    """Generate Morgan fingerprint for a molecule"""
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
    return np.array(fp)

def get_molecular_descriptors(mol):
    """Calculate basic molecular descriptors"""
    descriptors = {
        'MolWt': Descriptors.ExactMolWt(mol),
        'TPSA': Descriptors.TPSA(mol),
        'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumAromaticRings': Descriptors.NumAromaticRings(mol),
        'NumAliphaticRings': Descriptors.NumAliphaticRings(mol)
    }
    return descriptors

# Step 1: Data Collection and Initial Exploration

In [10]:

print("Loading dataset...")
df = pd.read_csv('logP_dataset.csv')

print("\nDataset Shape:", df.shape)
print("\nDataset Info:")
print(df.info())
print("\nLogP Statistics:")
print(df['LogP'].describe())

Loading dataset...

Dataset Shape: (14610, 2)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14610 entries, 0 to 14609
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SMILES  14610 non-null  object 
 1   LogP    14610 non-null  float64
dtypes: float64(1), object(1)
memory usage: 228.4+ KB
None

LogP Statistics:
count    14610.000000
mean         0.998665
std          1.300486
min         -3.600000
25%          0.100000
50%          1.000000
75%          1.800000
max          6.200000
Name: LogP, dtype: float64


# Step 2: Data Preprocessing

In [11]:

print("\nPreprocessing data...")
df['molecule'] = df['SMILES'].apply(validate_smiles)
df = df.dropna(subset=['molecule'])
df = df.drop_duplicates(subset=['SMILES'])

print(f"Number of valid molecules: {len(df)}")
print(f"Number of unique molecules: {df['SMILES'].nunique()}")


Preprocessing data...
Number of valid molecules: 14610
Number of unique molecules: 14610


In [12]:
df.head()

,SMILES,LogP,molecule
0,C[C@H]([C@@H](C)Cl)Cl,2.3,<rdkit.Chem.rdchem.Mol object at 0x14bee3d80>
1,C(C=CBr)N,0.3,<rdkit.Chem.rdchem.Mol object at 0x14bee3f40>
2,CCC(CO)Br,1.3,<rdkit.Chem.rdchem.Mol object at 0x14bee3ca0>
3,[13CH3][13CH2][13CH2][13CH2][13CH2][13CH2]O,2.0,<rdkit.Chem.rdchem.Mol object at 0x14bee3c30>
4,CCCOCCP,0.6,<rdkit.Chem.rdchem.Mol object at 0x14bee3bc0>


# Step 3: Feature Engineering

In [13]:

print("\nGenerating features...")
morgan_fps = np.array([get_morgan_fingerprint(mol) for mol in df['molecule']])
descriptors = pd.DataFrame([get_molecular_descriptors(mol) for mol in df['molecule']])

X = np.hstack([morgan_fps, descriptors])
y = df['LogP'].values

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)


Generating features...


[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerator
[15:24:50] DEPRECATION WARNING: please use MorganGenerat

Feature matrix shape: (14610, 1031)
Target vector shape: (14610,)


In [15]:
df2 = pd.DataFrame(X)  # Convert NumPy array to DataFrame
print(df2.head())

   0     1     2     3     4     5     6     7     8     9     ...  1021  \
0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   1022  1023        1024   1025  1026  1027  1028  1029  1030  
0   0.0   0.0  126.000306   0.00   1.0   0.0   0.0   0.0   0.0  
1   0.0   0.0  134.968361  26.02   1.0   1.0   1.0   0.0   0.0  
2   0.0   0.0  151.983677  20.23   2.0   1.0   1.0   0.0   0.0  
3   0.0   0.0  108.124594  20.23   4.0   1.0   1.0   0.0   0.0  
4   0.0   0.0  120.070402   9.23   4.0   1.0   0.0   0.0   0.0  

[5 rows x 1031 columns]


# Step 4: Model Training and Evaluation

In [16]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.2 MB/s eta 0:00:00a 0:00:010m
Note: you may need to restart the kernel to use updated packages.


In [21]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import joblib

In [22]:
# Save the scaler
joblib.dump(scaler, 'scaler.joblib')

# Train and save the best model (Random Forest)
best_model = RandomForestRegressor(n_estimators=100, random_state=42)
best_model.fit(X_train_scaled, y_train)
joblib.dump(best_model, 'model.joblib')

# Evaluate the best model
y_pred = best_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nBest Model (Random Forest) Results:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R2: {r2:.4f}")


Best Model (Random Forest) Results:
RMSE: 0.3703
MAE: 0.2289
R2: 0.9202


In [18]:

# print("\nTraining models...")
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# models = {
#     'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
#     'XGBoost': xgb.XGBRegressor(random_state=42),
#     'SVR': SVR(kernel='rbf')
# }

# results = {}
# for name, model in models.items():
#     print(f"\nTraining {name}...")
#     model.fit(X_train_scaled, y_train)
#     y_pred = model.predict(X_test_scaled)
    
#     mse = mean_squared_error(y_test, y_pred)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)
    
#     results[name] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
#     print(f"{name} Results:")
#     print(f"RMSE: {rmse:.4f}")
#     print(f"MAE: {mae:.4f}")
#     print(f"R2: {r2:.4f}")



Training models...

Training Random Forest...
Random Forest Results:
RMSE: 0.3703
MAE: 0.2289
R2: 0.9202

Training XGBoost...
XGBoost Results:
RMSE: 0.3307
MAE: 0.2243
R2: 0.9364

Training SVR...
SVR Results:
RMSE: 0.5262
MAE: 0.3540
R2: 0.8389


# Step 5: Visualization and Analysis


In [23]:
print("\nGenerating visualizations...")

# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual LogP')
plt.ylabel('Predicted LogP')
plt.title('Actual vs Predicted LogP Values')
plt.savefig('actual_vs_predicted.png')
plt.close()

# Feature importance
feature_importance = pd.DataFrame({
    'feature': [f'Morgan_{i}' for i in range(morgan_fps.shape[1])] + list(descriptors.columns),
    'importance': best_model.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('Top 20 Most Important Features')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

print("\nModel and visualizations have been saved.")


Generating visualizations...

Model and visualizations have been saved.


In [19]:

# plt.figure(figsize=(10, 6))
# for name, model in models.items():
#     y_pred = model.predict(X_test_scaled)
#     plt.scatter(y_test, y_pred, alpha=0.5, label=name)

# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
# plt.xlabel('Actual LogP')
# plt.ylabel('Predicted LogP')
# plt.title('Actual vs Predicted LogP Values')
# plt.legend()
# plt.savefig('actual_vs_predicted.png')
# plt.close()

# Feature importance for Random Forest

In [20]:

# rf_model = models['Random Forest']
# feature_importance = pd.DataFrame({
#     'feature': [f'Morgan_{i}' for i in range(morgan_fps.shape[1])] + list(descriptors.columns),
#     'importance': rf_model.feature_importances_
# })
# feature_importance = feature_importance.sort_values('importance', ascending=False)

# plt.figure(figsize=(12, 6))
# sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
# plt.title('Top 20 Most Important Features')
# plt.tight_layout()
# plt.savefig('feature_importance.png')
# plt.close()

# print("\nVisualizations have been saved as 'actual_vs_predicted.png' and 'feature_importance.png'")


Visualizations have been saved as 'actual_vs_predicted.png' and 'feature_importance.png'
